### Dependencies

In [2]:
# !pip install pypdf langchain langchain_core langchain_community langchain_huggingface langchain_groq sentence_transformers transformers langchain_google_genai faiss-cpu langchain_openai grandalf pymupdf 

### Loading PDF

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("GSTsmartGuide.pdf")

In [5]:
docs = loader.load()
docs[0]

Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-02-27T11:53:51+05:30', 'source': 'GSTsmartGuide.pdf', 'file_path': 'GSTsmartGuide.pdf', 'total_pages': 1321, 'format': 'PDF 1.6', 'title': 'CHAPTER 1', 'author': 'Abha', 'subject': '', 'keywords': '', 'moddate': '2024-02-27T13:03:27+05:30', 'trapped': '', 'modDate': "D:20240227130327+05'30'", 'creationDate': "D:20240227115351+05'30'", 'page': 0}, page_content='')

In [6]:
len(docs)

1321

In [7]:
docs[100].page_content

'84 \nGST Smart Guide \nChap. 5 \n \n(3) The place of supply of the following services shall be the location where \nthe services are actually performed, namely:—  \n(a) services supplied in respect of goods which are required to be made \nphysically available by the recipient of services to the supplier of \nservices, or to a person acting on behalf of the supplier of services in \norder to provide the services:  \n \nProvided that when such services are provided from a remote location by \nway of electronic means, the place of supply shall be the location where \ngoods are situated at the time of supply of services: \n \nProvided further that nothing contained in this clause shall apply in the \ncase of services supplied in respect of goods which are temporarily \nimported into India for repairs and are exported after repairs without \nbeing put to any other use in India, than that which is required for such \nrepairs; \n(b) services supplied to an individual, represented either as t

## Late Chunking

In [8]:
from transformers import AutoModel
from transformers import AutoTokenizer

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

configuration_bert.py:   0%|          | 0.00/8.24k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_bert.py:   0%|          | 0.00/97.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- modeling_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/275M [00:00<?, ?B/s]

Now we define the text which we want to encode and split it into chunks. The `chunk_by_sentences` function also returns the span annotations.
Those specify the number of tokens per chunk which is needed for the chunked pooling.

In [9]:
def chunk_by_sentences(input_text: str, tokenizer: callable):
    """
    Split the input text into sentences using the tokenizer
    :param input_text: The text snippet to split into sentences
    :param tokenizer: The tokenizer to use
    :return: A tuple containing the list of text chunks and their corresponding token spans
    """
    inputs = tokenizer(input_text, return_tensors='pt', return_offsets_mapping=True)
    punctuation_mark_id = tokenizer.convert_tokens_to_ids('.')
    sep_id = tokenizer.convert_tokens_to_ids('[SEP]')
    token_offsets = inputs['offset_mapping'][0]
    token_ids = inputs['input_ids'][0]
    chunk_positions = [
        (i, int(start + 1))
        for i, (token_id, (start, end)) in enumerate(zip(token_ids, token_offsets))
        if token_id == punctuation_mark_id
        and (
            token_offsets[i + 1][0] - token_offsets[i][1] > 0
            or token_ids[i + 1] == sep_id
        )
    ]
    chunks = [
        input_text[x[1] : y[1]]
        for x, y in zip([(1, 0)] + chunk_positions[:-1], chunk_positions)
    ]
    span_annotations = [
        (x[0], y[0]) for (x, y) in zip([(1, 0)] + chunk_positions[:-1], chunk_positions)
    ]
    return chunks, span_annotations

In [11]:
content=[]
for doc in docs:
  content.append(doc.page_content)

In [12]:
len(content)

1321

In [25]:
content[100]

'84 \nGST Smart Guide \nChap. 5 \n \n(3) The place of supply of the following services shall be the location where \nthe services are actually performed, namely:—  \n(a) services supplied in respect of goods which are required to be made \nphysically available by the recipient of services to the supplier of \nservices, or to a person acting on behalf of the supplier of services in \norder to provide the services:  \n \nProvided that when such services are provided from a remote location by \nway of electronic means, the place of supply shall be the location where \ngoods are situated at the time of supply of services: \n \nProvided further that nothing contained in this clause shall apply in the \ncase of services supplied in respect of goods which are temporarily \nimported into India for repairs and are exported after repairs without \nbeing put to any other use in India, than that which is required for such \nrepairs; \n(b) services supplied to an individual, represented either as t

In [17]:
def chunk_by_sentences(input_text, tokenizer: callable):
    """
    Split the input text into sentences using the tokenizer
    :param input_text: The text snippet to split into sentences (string)
    :param tokenizer: The tokenizer to use
    :return: A tuple containing the list of text chunks and their corresponding token spans
    """
    # Ensure input_text is a string (e.g., if given a list with one string inside)
    if isinstance(input_text, list):
        input_text = " ".join(input_text)

    # Tokenize the input
    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        return_offsets_mapping=True,
        padding=True,
        truncation=True
    )

    punctuation_mark_id = tokenizer.convert_tokens_to_ids('.')
    sep_id = tokenizer.convert_tokens_to_ids('[SEP]')
    token_offsets = inputs['offset_mapping'][0]
    token_ids = inputs['input_ids'][0]

    chunk_positions = [
        (i, int(start + 1))
        for i, (token_id, (start, end)) in enumerate(zip(token_ids, token_offsets))
        if token_id == punctuation_mark_id
        and (
            token_offsets[i + 1][0] - token_offsets[i][1] > 0
            or token_ids[i + 1] == sep_id
        )
    ]

    chunks = [
        input_text[x[1]: y[1]]
        for x, y in zip([(1, 0)] + chunk_positions[:-1], chunk_positions)
    ]
    span_annotations = [
        (x[0], y[0]) for (x, y) in zip([(1, 0)] + chunk_positions[:-1], chunk_positions)
    ]

    return chunks, span_annotations


In [ ]:
chunks, spans = chunk_by_sentences(content[100], tokenizer)

In [19]:
def late_chunking(
    model_output: 'BatchEncoding', span_annotation: list, max_length=None
):
    token_embeddings = model_output[0]
    outputs = []
    for embeddings, annotations in zip(token_embeddings, span_annotation):
        if (
            max_length is not None
        ):  # remove annotations which go bejond the max-length of the model
            annotations = [
                (start, min(end, max_length - 1))
                for (start, end) in annotations
                if start < (max_length - 1)
            ]
        pooled_embeddings = [
            embeddings[start:end].sum(dim=0) / (end - start)
            for start, end in annotations
            if (end - start) >= 1
        ]
        pooled_embeddings = [
            embedding.detach().cpu().numpy() for embedding in pooled_embeddings
        ]
        outputs.append(pooled_embeddings)

    return outputs

In [23]:
# chunk before
embeddings_traditional_chunking = model.encode(chunks)

# chunk afterwards (context-sensitive chunked pooling)
inputs = tokenizer(content[100], return_tensors='pt')
model_output = model(**inputs)
embeddings = late_chunking(model_output, [spans])[0]

In [28]:
import numpy as np

cos_sim = lambda x, y: np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

late_embedding = model.encode("How is place of supply determined for prepaid services?")

for chunk, new_embedding, trad_embeddings in zip(chunks, embeddings, embeddings_traditional_chunking):
    print(f'similarity_new("{chunk}"):', cos_sim(late_embedding, new_embedding))
    print(f'similarity_trad( "{chunk}"):', cos_sim(late_embedding, trad_embeddings))

similarity_new("84 
GST Smart Guide 
Chap."): 0.55179465
similarity_trad( "84 
GST Smart Guide 
Chap."): 0.690149
similarity_new(" 5 
 
(3) The place of supply of the following services shall be the location where 
the services are actually performed, namely:—  
(a) services supplied in respect of goods which are required to be made 
physically available by the recipient of services to the supplier of 
services, or to a person acting on behalf of the supplier of services in 
order to provide the services:  
 
Provided that when such services are provided from a remote location by 
way of electronic means, the place of supply shall be the location where 
goods are situated at the time of supply of services: 
 
Provided further that nothing contained in this clause shall apply in the 
case of services supplied in respect of goods which are temporarily 
imported into India for repairs and are exported after repairs without 
being put to any other use in India, than that which is required 